<a href="https://colab.research.google.com/github/bellazeng2016/playground/blob/main/Recommender_Sys_Retrieval_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pprint
import tempfile
from typing import Dict, Text
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [1]:
import tensorflow_recommenders as tfrs

In [3]:
ratings = tfds.load("movielens/100k-ratings", split='train')
movies = tfds.load("movielens/100k-movies", split='train')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteNH72GM/movielens-train.tfrecord*...:   0…

Dataset movielens downloaded and prepared to ~/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [5]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [6]:
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [7]:
ratings = ratings.map(
    lambda x: {
        "movie_title": x['movie_title'],
        "user_id": x['user_id']
    }
)
movies = movies.map(
    lambda x: x['movie_title']
)

In [13]:
tfds.as_dataframe(movies).shape

(1682, 1)

In [12]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration= False)
train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [14]:
movie_titles = movies.batch(1_000)

In [16]:
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

In [20]:
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [22]:
unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

In [23]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary = unique_user_ids, mask_token= None
    ),
    tf.keras.layers.Embedding(len(unique_user_ids)+1, embedding_dimension)
])